The same as "pytorch_training_loop" but better :)

In [1]:
import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from model import *
from data import *
from fastai.text.all import *

In [3]:
hparams = argparse.Namespace(**{'load': False,
                                'window': 4096,
                                'step': 4096,
                                'classify': True,
                                'tgt_tax_lvl': "phylum",
                                'fwd_only': True})

In [4]:
def get_toy_dl(hparams, batch_size=16):
    path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
    chunks = LazySeqDataset(hparams, path=path,
                           keep_open=True)
    ds = taxon_ds(chunks, old_pad_seq)
    return DataLoader(ds, batch_size=batch_size, 
                      shuffle=True), ds

In [5]:
dl,ds = get_toy_dl(hparams, batch_size=128)
batch = next(iter(dl))
len(dl), batch[0].shape, batch[1].shape

(149, torch.Size([128, 1, 4096]), torch.Size([128]))

In [24]:
def get_model():
    model = nn.Sequential(
    get_base_layer(),
    get_dep_sep(in_ch=32,out_ch=16),
    get_inv_res(in_ch=16, mid_ch=96, out_ch= 24, 
                sq_ch=4, ks=3, stride=2, padding=1),
    get_head_layer(320, 1,
                    lin_out_feats=18))
    return model

In [25]:
#model = get_model()
model = get_eff_b0(18)

In [26]:
dls = DataLoaders(dl, dl)

In [27]:
#get_head_layer

As a reminder, we're just playing around here -- a proper validation set should be used and not just the same dataloader as the training set :)

In [28]:
learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss())

Trying to train this model will not work -- out of the box -- this can be confirmed by uncommenting the line below

In [9]:
#learn.fit(1)

We'll need to utilize fast.ai's callback system in order to make sure out training loop includes that dimension reduction with the squeeze function, on our models output -- before it get's fed into our loss function. You can find out more about callbacks here: https://docs.fast.ai/callback.core.html

In [29]:
class compress_cb(Callback):
    def after_pred(self):
        self.learn.pred = self.pred.squeeze(1)

In [30]:
dls.to('cuda');
model.to('cuda');

In [31]:
learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss(),
               cbs=[compress_cb], metrics=[accuracy]).to_fp16()

In [32]:
learn.lr_find()

ValueError: Expected target size (128, 1), got torch.Size([128])

In [24]:
learn.fit_one_cycle(2, 1e-1)

epoch,train_loss,valid_loss,accuracy,time
0,2.175484,2.110091,0.316518,05:52
1,2.102375,2.065571,0.320358,05:53


That was super easy! With Fast.AI we can access useful things like the learning rate finder: https://fastai1.fast.ai/callbacks.lr_finder.html: https://arxiv.org/abs/1506.01186

We also get a ton of other useful functionality, like tables for our results + tools to inspect/diagnose how well our model is doing + easy to use techniques to train our model -- for example: <br>
(1) using mixed precision training only requires the ".to_fp16()" as seen in our Learner call<br>
(2) One cycle training is implemented in our fit call above

In [79]:
#gpu is clearly available
#!nvidia-smi

Need to look into why library is having trouble assigning everything to gpu automatically -- perhaps bc using cluster?